In [1]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData, Integer, String
import pandas as pd

In [2]:
from sqlalchemy.sql import func, and_, desc, alias, text, and_, cast

In [3]:
pd.options.display.max_rows=200

In [4]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [5]:
conn = engine.connect()

In [6]:
metadata = db.MetaData(bind=engine)

#### Test der Abfrage für eine einzelne Tabelle

In [7]:
table = db.Table('tcdata_CH_0084_01', metadata, autoload = True)

#### Intro eines Alias für table (WICHTIG: *Alias* gilt nur für Tabellen. **Bemerkung:** Spalten kann man mit *Labels* umbenennen...

In [8]:
t = table.alias()

In [9]:
q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), \
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
where(t.c.vfr11 >0).\
group_by(func.strftime('%H', t.c.time)). \
order_by('Wochentag')
result = conn.execute(q)
row = result.fetchall()

In [10]:
df_avg_gesamt= pd.read_sql_query(q, conn)
df_avg_gesamt.head(12)

,time,Anzahl,Jahr,Monat,Woche,Tag,Stunde,Wochentag,AVG_flow,AVG_speed
0,2021-07-27 19:53:00+02:00,6703,2021,07,30,27,19,2,1312.0,64.0
1,2021-07-27 20:03:00+02:00,6730,2021,07,30,27,20,2,1019.0,66.0
2,2021-07-27 21:03:00+02:00,6466,2021,07,30,27,21,2,822.0,67.0
3,2021-07-27 22:08:00+02:00,5223,2021,07,30,27,22,2,702.0,66.0
4,2021-07-27 23:03:00+02:00,4660,2021,07,30,27,23,2,572.0,66.0
5,2021-07-28 00:03:00+02:00,4486,2021,07,30,27,00,2,401.0,66.0
6,2021-07-28 01:03:00+02:00,4113,2021,07,30,27,01,2,257.0,66.0
7,2021-07-28 02:03:00+02:00,3596,2021,07,30,28,02,3,185.0,66.0
8,2021-07-28 03:03:00+02:00,3426,2021,07,30,28,03,3,149.0,66.0
9,2021-07-28 04:03:00+02:00,3951,2021,07,30,28,04,3,165.0,67.0


#### Ok, damit kann ich den Loop vorbereiten, mit dem ich die DFs erstelle, die schliesslich als csv-Dateien abgespeichert werden

In [11]:
inspector = inspect(engine)

In [12]:
tab = inspector.get_table_names()
tab = tab[:-1]
len(tab)

75

In [13]:
tab

['tcdata_CH_0020_01',
 'tcdata_CH_0020_02',
 'tcdata_CH_0020_03',
 'tcdata_CH_0020_04',
 'tcdata_CH_0020_05',
 'tcdata_CH_0020_06',
 'tcdata_CH_0020_07',
 'tcdata_CH_0020_08',
 'tcdata_CH_0084_01',
 'tcdata_CH_0084_02',
 'tcdata_CH_0084_03',
 'tcdata_CH_0084_04',
 'tcdata_CH_0150_01',
 'tcdata_CH_0150_02',
 'tcdata_CH_0240_01',
 'tcdata_CH_0240_02',
 'tcdata_CH_0240_03',
 'tcdata_CH_0240_04',
 'tcdata_CH_0240_05',
 'tcdata_CH_0240_06',
 'tcdata_CH_0240_07',
 'tcdata_CH_0240_08',
 'tcdata_CH_0243_01',
 'tcdata_CH_0243_02',
 'tcdata_CH_0243_03',
 'tcdata_CH_0243_04',
 'tcdata_CH_0274_01',
 'tcdata_CH_0274_02',
 'tcdata_CH_0274_03',
 'tcdata_CH_0274_04',
 'tcdata_CH_0274_05',
 'tcdata_CH_0274_06',
 'tcdata_CH_0282_01',
 'tcdata_CH_0282_02',
 'tcdata_CH_0282_03',
 'tcdata_CH_0282_04',
 'tcdata_CH_0282_05',
 'tcdata_CH_0282_06',
 'tcdata_CH_0286_01',
 'tcdata_CH_0286_02',
 'tcdata_CH_0286_03',
 'tcdata_CH_0286_04',
 'tcdata_CH_0286_05',
 'tcdata_CH_0286_06',
 'tcdata_CH_0286_07',
 'tcdata_C

In [14]:
import os
os.getcwd()

'C:\\Users\\peter\\Documents\\Projekte\\Coding\\Jupyter\\RosBe'

#### I. Hier starte ich jetzt mit den Auswertungen (Durchschnitte) pro Stunde, die ich mache für: 
<ol>
    <li>den Gesamtzeitraum</li>
    <li>pro Monat</li>
    <li>pro Woche</li>
    <li>pro Wochentag</li>
</ol>



#### Gesamtzeitraum

In [91]:
ListAnzahl = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    ListAnzahl.append(table[10:])
    
    q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
    where(t.c.vfr11 >0).\
    group_by(func.strftime('%H', t.c.time)). \
    order_by('Stunde')
    
    result = conn.execute(q)
    row = result.fetchall()
    ListAnzahl.append(len(row)) 
    df = pd.read_sql_query(q, conn)
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\gesamt_avg_' + table[10:] + '.csv', index = False)

In [92]:
print(len(row))

24


In [93]:
df_avg_monat= pd.read_sql_query(q, conn)
df_avg_monat.head(14)

,time,Anzahl,Jahr,Monat,Woche,Tag,Stunde,Minute,Wochentag,AVG_flow,AVG_speed
0,2021-07-28 00:28:00+02:00,2218,2021,07,30,27,00,28,2,86.0,60.0
1,2021-07-28 01:38:00+02:00,1603,2021,07,30,27,01,38,2,78.0,60.0
2,2021-07-28 02:28:00+02:00,1335,2021,07,30,28,02,28,3,74.0,59.0
3,2021-07-28 03:18:00+02:00,1291,2021,07,30,28,03,18,3,73.0,59.0
4,2021-07-28 04:23:00+02:00,1498,2021,07,30,28,04,23,3,75.0,58.0
5,2021-07-28 05:38:00+02:00,3447,2021,07,30,28,05,38,3,91.0,59.0
6,2021-07-28 06:01:00+02:00,17910,2021,07,30,28,06,01,3,124.0,59.0
7,2021-07-28 07:00:00+02:00,22438,2021,07,30,28,07,00,3,142.0,60.0
8,2021-07-28 08:01:00+02:00,22205,2021,07,30,28,08,01,3,138.0,59.0
9,2021-07-28 09:02:00+02:00,5099,2021,07,30,28,09,02,3,129.0,60.0


#### pro Monat

In [94]:
ListAnzahl = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    ListAnzahl.append(table[10:])
    
    q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
    where(t.c.vfr11 >0).\
    group_by(func.strftime('%H', t.c.time), func.strftime('%m', t.c.time)). \
    order_by('Stunde')
    
    result = conn.execute(q)
    row = result.fetchall()
    ListAnzahl.append(len(row)) 
    df = pd.read_sql_query(q, conn)
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\monat_avg_' + table[10:] + '.csv', index = False)

In [95]:
print(len(row))

288


#### pro Woche

In [96]:
ListAnzahl = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    ListAnzahl.append(table[10:])
    
    q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
    where(t.c.vfr11 >0).\
    group_by(func.strftime('%H', t.c.time), func.strftime('%W', t.c.time)). \
    order_by('Stunde')
    
    result = conn.execute(q)
    row = result.fetchall()
    ListAnzahl.append(len(row)) 
    df = pd.read_sql_query(q, conn)
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\woche_avg_' + table[10:] + '.csv', index = False)

In [97]:
print(len(row))

1272


#### pro Wochentag

In [98]:
ListAnzahl = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    ListAnzahl.append(table[10:])
    
    q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
    where(t.c.vfr11 >0).\
    group_by(func.strftime('%H', t.c.time), func.strftime('%w', t.c.time)). \
    order_by('Stunde')
    
    result = conn.execute(q)
    row = result.fetchall()
    ListAnzahl.append(len(row)) 
    df = pd.read_sql_query(q, conn)
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\wotag_avg_' + table[10:] + '.csv', index = False)

In [99]:
print(len(row))

168


#### Nun folgen Abfragen, die dazu dienen, die Konturen von Flow und Speed im Verlauf eines Tages zu zeigen. Dabei werden Dateien in der Frequenz von 1h, 15min und 1min (für die Stosszeiten) generiert

#### Ein Basisabfrage für 1h (vor dem Loop) sieht zB so aus

In [89]:
q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round((t.c.vfr11)).label('flow'), func.round((t.c.s12)).label('speed')). \
filter(cast(func.strftime('%H', t.c.time, 'localtime'), Integer) == 23)
result = conn.execute(q)
row = result.fetchall()

In [90]:
df_test= pd.read_sql_query(q, conn)
df_test.head()

,time,Jahr,Monat,Woche,Tag,Stunde,Minute,Wochentag,flow,speed
0,2021-07-27 23:03:00+02:00,2021,07,30,27,23,03,2,0.0,0.0
1,2021-07-27 23:08:00+02:00,2021,07,30,27,23,08,2,60.0,56.0
2,2021-07-27 23:13:00+02:00,2021,07,30,27,23,13,2,0.0,0.0
3,2021-07-27 23:18:00+02:00,2021,07,30,27,23,18,2,0.0,0.0
4,2021-07-27 23:23:00+02:00,2021,07,30,27,23,23,2,0.0,0.0


#### II. Loop über alle Tabellen für Kontouren

#### Average pro Stunde

In [50]:
len_tab = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    df = pd.DataFrame()
    df_list = []
    for h in range(24):
        q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
               func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')).  \
        filter(and_(t.c.vfr11 >= 0, cast(func.strftime('%H', t.c.time, 'localtime'), Integer) == h))
        result = conn.execute(q)
        row = result.fetchall()
        df_q = pd.read_sql_query(q, conn)
        df_list.append(df_q)
    df = pd.concat(df_list, ignore_index=True)
    len_tab.append(table)
    len_tab.append(len(df))
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\Bucket_Stunde_AVG_' +  table[10:] + '.csv', index = False)

In [52]:
len_tab

['tcdata_CH_0020_01', 23, 24, 'tcdata_CH_0020_02', 23, 24]

#### Average pro 15min

In [51]:
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    df = pd.DataFrame()
    df_list = []
    print(table)
    for h in range(24):     
        for min in range(0, 60, 15):
            q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
                  func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
                   func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')).  \
            filter(and_(t.c.vfr11 >= 0, cast(func.strftime('%H', t.c.time, 'localtime'), Integer) == h), cast(func.strftime('%M', t.c.time, 'localtime'), Integer) < (min+15), cast(func.strftime('%M', t.c.time, 'localtime'), Integer) >=  min)
            result = conn.execute(q)
            row = result.fetchall()
            df_q = pd.read_sql_query(q, conn)
            df_list.append(df_q)
            df = pd.concat(df_list, ignore_index=True)  
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\Bucket_15min_AVG_' + table[10:] + '.csv', index = False)

tcdata_CH_0020_01
tcdata_CH_0020_02
tcdata_CH_0020_03
tcdata_CH_0020_04
tcdata_CH_0020_05
tcdata_CH_0020_06
tcdata_CH_0020_07
tcdata_CH_0020_08
tcdata_CH_0084_01
tcdata_CH_0084_02
tcdata_CH_0084_03
tcdata_CH_0084_04
tcdata_CH_0150_01
tcdata_CH_0150_02
tcdata_CH_0240_01
tcdata_CH_0240_02
tcdata_CH_0240_03
tcdata_CH_0240_04
tcdata_CH_0240_05
tcdata_CH_0240_06
tcdata_CH_0240_07
tcdata_CH_0240_08
tcdata_CH_0243_01
tcdata_CH_0243_02
tcdata_CH_0243_03
tcdata_CH_0243_04
tcdata_CH_0274_01
tcdata_CH_0274_02
tcdata_CH_0274_03
tcdata_CH_0274_04
tcdata_CH_0274_05
tcdata_CH_0274_06
tcdata_CH_0282_01
tcdata_CH_0282_02
tcdata_CH_0282_03
tcdata_CH_0282_04
tcdata_CH_0282_05
tcdata_CH_0282_06
tcdata_CH_0286_01
tcdata_CH_0286_02
tcdata_CH_0286_03
tcdata_CH_0286_04
tcdata_CH_0286_05
tcdata_CH_0286_06
tcdata_CH_0286_07
tcdata_CH_0286_08
tcdata_CH_0287_01
tcdata_CH_0287_02
tcdata_CH_0287_03
tcdata_CH_0287_04
tcdata_CH_0528_01
tcdata_CH_0528_02
tcdata_CH_0528_03
tcdata_CH_0528_04
tcdata_CH_0529_01
tcdata_CH_

['tcdata_CH_0020_01',
 96,
 'tcdata_CH_0020_02',
 96,
 'tcdata_CH_0020_03',
 96,
 'tcdata_CH_0020_04',
 96,
 'tcdata_CH_0020_05',
 96,
 'tcdata_CH_0020_06',
 96,
 'tcdata_CH_0020_07',
 96,
 'tcdata_CH_0020_08',
 96,
 'tcdata_CH_0084_01',
 96,
 'tcdata_CH_0084_02',
 96,
 'tcdata_CH_0084_03',
 96,
 'tcdata_CH_0084_04',
 96,
 'tcdata_CH_0150_01',
 96,
 'tcdata_CH_0150_02',
 96,
 'tcdata_CH_0240_01',
 96,
 'tcdata_CH_0240_02',
 96,
 'tcdata_CH_0240_03',
 96,
 'tcdata_CH_0240_04',
 96,
 'tcdata_CH_0240_05',
 96,
 'tcdata_CH_0240_06',
 96,
 'tcdata_CH_0240_07',
 96,
 'tcdata_CH_0240_08',
 96,
 'tcdata_CH_0243_01',
 96,
 'tcdata_CH_0243_02',
 96,
 'tcdata_CH_0243_03',
 96,
 'tcdata_CH_0243_04',
 96,
 'tcdata_CH_0274_01',
 96,
 'tcdata_CH_0274_02',
 96,
 'tcdata_CH_0274_03',
 96,
 'tcdata_CH_0274_04',
 96,
 'tcdata_CH_0274_05',
 96,
 'tcdata_CH_0274_06',
 96,
 'tcdata_CH_0282_01',
 96,
 'tcdata_CH_0282_02',
 96,
 'tcdata_CH_0282_03',
 96,
 'tcdata_CH_0282_04',
 96,
 'tcdata_CH_0282_05',
 96,
 

#### Hier generiere ich Dateien mit den Basisdaten für alle Zeitpunkte, um später die AVG prüfen zu können

In [52]:
len_tab = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    df = pd.DataFrame()
    print(table)
    for h in range(24):     
        for min in range(0, 60, 15):
            q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
                  func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
                   func.strftime('%w', t.c.time).label('Wochentag'), func.round(t.c.vfr11).label('flow'), func.round(t.c.s12).label('speed')).  \
            filter(and_(t.c.vfr11 >= 0, cast(func.strftime('%H', t.c.time, 'localtime'), Integer) == h), cast(func.strftime('%M', t.c.time, 'localtime'), Integer) < (min+15), cast(func.strftime('%M', t.c.time, 'localtime'), Integer) >= min)
            result = conn.execute(q)
            row = result.fetchall()
            df_q = pd.read_sql_query(q, conn)
            len_tab.append(table)
            len_tab.append(len(df))
            df_q.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\Bucket_15min_basis_'  + str(h) + '_'  + str(min) + '_' + table[10:] + '.csv', index = False)

tcdata_CH_0020_01
tcdata_CH_0020_02
tcdata_CH_0020_03
tcdata_CH_0020_04
tcdata_CH_0020_05
tcdata_CH_0020_06
tcdata_CH_0020_07
tcdata_CH_0020_08
tcdata_CH_0084_01
tcdata_CH_0084_02
tcdata_CH_0084_03
tcdata_CH_0084_04
tcdata_CH_0150_01
tcdata_CH_0150_02
tcdata_CH_0240_01
tcdata_CH_0240_02
tcdata_CH_0240_03
tcdata_CH_0240_04
tcdata_CH_0240_05
tcdata_CH_0240_06
tcdata_CH_0240_07
tcdata_CH_0240_08
tcdata_CH_0243_01
tcdata_CH_0243_02
tcdata_CH_0243_03
tcdata_CH_0243_04
tcdata_CH_0274_01
tcdata_CH_0274_02
tcdata_CH_0274_03
tcdata_CH_0274_04
tcdata_CH_0274_05
tcdata_CH_0274_06
tcdata_CH_0282_01
tcdata_CH_0282_02
tcdata_CH_0282_03
tcdata_CH_0282_04
tcdata_CH_0282_05
tcdata_CH_0282_06
tcdata_CH_0286_01
tcdata_CH_0286_02
tcdata_CH_0286_03
tcdata_CH_0286_04
tcdata_CH_0286_05
tcdata_CH_0286_06
tcdata_CH_0286_07
tcdata_CH_0286_08
tcdata_CH_0287_01
tcdata_CH_0287_02
tcdata_CH_0287_03
tcdata_CH_0287_04
tcdata_CH_0528_01
tcdata_CH_0528_02
tcdata_CH_0528_03
tcdata_CH_0528_04
tcdata_CH_0529_01
tcdata_CH_

#### Average pro min zu Stosszeiten

In [56]:
len_tab = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    df = pd.DataFrame()
    df_list = []
    print(table)
    for h in {6, 7, 8, 16, 17, 18}:     
        for min in range(60):
            q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
                  func.strftime('%W', t.c.time).label('Woche'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), func.strftime('%M', t.c.time, 'localtime').label('Minute'),\
                   func.strftime('%w', t.c.time).label('Wochentag'), func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')).  \
            filter(and_(t.c.vfr11 >= 0, cast(func.strftime('%H', t.c.time, 'localtime'), Integer) == h), cast(func.strftime('%M', t.c.time, 'localtime'), Integer) == min)
            result = conn.execute(q)
            row = result.fetchall()
            df_q = pd.read_sql_query(q, conn)
            df_list.append(df_q)
            df = pd.concat(df_list, ignore_index=True)  
    len_tab.append(table)
    len_tab.append(len(df))
    df.to_csv(r'C:\Users\peter\Documents\Projekte\Coding\Jupyter\Grafiken\Bucket_min_RushHour_AVG_' + table[10:] + '.csv', index = False)
len_tab

tcdata_CH_0020_01
tcdata_CH_0020_02
tcdata_CH_0020_03
tcdata_CH_0020_04
tcdata_CH_0020_05
tcdata_CH_0020_06
tcdata_CH_0020_07
tcdata_CH_0020_08
tcdata_CH_0084_01
tcdata_CH_0084_02
tcdata_CH_0084_03
tcdata_CH_0084_04
tcdata_CH_0150_01
tcdata_CH_0150_02
tcdata_CH_0240_01
tcdata_CH_0240_02
tcdata_CH_0240_03
tcdata_CH_0240_04
tcdata_CH_0240_05
tcdata_CH_0240_06
tcdata_CH_0240_07
tcdata_CH_0240_08
tcdata_CH_0243_01
tcdata_CH_0243_02
tcdata_CH_0243_03
tcdata_CH_0243_04
tcdata_CH_0274_01
tcdata_CH_0274_02
tcdata_CH_0274_03
tcdata_CH_0274_04
tcdata_CH_0274_05
tcdata_CH_0274_06
tcdata_CH_0282_01
tcdata_CH_0282_02
tcdata_CH_0282_03
tcdata_CH_0282_04
tcdata_CH_0282_05
tcdata_CH_0282_06
tcdata_CH_0286_01
tcdata_CH_0286_02
tcdata_CH_0286_03
tcdata_CH_0286_04
tcdata_CH_0286_05
tcdata_CH_0286_06
tcdata_CH_0286_07
tcdata_CH_0286_08
tcdata_CH_0287_01
tcdata_CH_0287_02
tcdata_CH_0287_03
tcdata_CH_0287_04
tcdata_CH_0528_01
tcdata_CH_0528_02
tcdata_CH_0528_03
tcdata_CH_0528_04
tcdata_CH_0529_01
tcdata_CH_

['tcdata_CH_0020_01',
 360,
 'tcdata_CH_0020_02',
 360,
 'tcdata_CH_0020_03',
 360,
 'tcdata_CH_0020_04',
 360,
 'tcdata_CH_0020_05',
 360,
 'tcdata_CH_0020_06',
 360,
 'tcdata_CH_0020_07',
 360,
 'tcdata_CH_0020_08',
 360,
 'tcdata_CH_0084_01',
 360,
 'tcdata_CH_0084_02',
 360,
 'tcdata_CH_0084_03',
 360,
 'tcdata_CH_0084_04',
 360,
 'tcdata_CH_0150_01',
 360,
 'tcdata_CH_0150_02',
 360,
 'tcdata_CH_0240_01',
 360,
 'tcdata_CH_0240_02',
 360,
 'tcdata_CH_0240_03',
 360,
 'tcdata_CH_0240_04',
 360,
 'tcdata_CH_0240_05',
 360,
 'tcdata_CH_0240_06',
 360,
 'tcdata_CH_0240_07',
 360,
 'tcdata_CH_0240_08',
 360,
 'tcdata_CH_0243_01',
 360,
 'tcdata_CH_0243_02',
 360,
 'tcdata_CH_0243_03',
 360,
 'tcdata_CH_0243_04',
 360,
 'tcdata_CH_0274_01',
 360,
 'tcdata_CH_0274_02',
 360,
 'tcdata_CH_0274_03',
 360,
 'tcdata_CH_0274_04',
 360,
 'tcdata_CH_0274_05',
 360,
 'tcdata_CH_0274_06',
 360,
 'tcdata_CH_0282_01',
 360,
 'tcdata_CH_0282_02',
 360,
 'tcdata_CH_0282_03',
 360,
 'tcdata_CH_0282_04'